In [83]:
# Imports
import pandas as pd
import plotly.express as px
from prettytable import PrettyTable
import plotly.graph_objects as go
import numpy as np
import random
import csv
import os
import json
import altair as alt
from tqdm import tqdm
import numpy as np
import seaborn as sns
#import dataframe_image as dfi
from numpy.random import seed
from numpy.random import randn
from statsmodels.graphics.gofplots import qqplot
from matplotlib import pyplot
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [84]:
WD = os.path.join("E:\\", "BUSTED_ModelTest")

#WD = "/Users/alex/Documents/BUSTEDS-MH"
DATA = os.path.join(WD, "tables", "Table_14-datasets_results_adjusted.csv")

In [85]:
print("Reading:", DATA)
df = pd.read_csv(DATA)
df = df.reset_index(drop=True)
df.index += 1
df

Reading: E:\BUSTED_ModelTest\tables\Table_14-datasets_results_adjusted.csv


,Gene,Method,Sequences,Codons,LRT p-value,Bonferroni p-value,Bonferroni significant,cAIC,CV(omega),CV(alpha),...,ER_SITES_CONSTRAINED,NUM_ER_SITES_OPTIMIZED_NULL,ER_SITES_OPTIMIZED_NULL,NUM_ER_SITES_CONSTRAINED_loose,ER_SITES_CONSTRAINED_loose,NUM_ER_SITES_OPTIMIZED_NULL_loose,ER_SITES_OPTIMIZED_NULL_loose,DH_Rate,TH_Rate,TH_Rate_SI
1,COXI,BUSTED,21,510,5.000000e-01,0.0125,False,24384.439201,0.839072,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,COXI,BUSTED-MH,21,510,5.000000e-01,0.0125,False,24389.451070,0.814080,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,5.676599
3,COXI,BUSTEDS,21,510,5.000000e-01,0.0125,False,24288.101212,1.381566,1.330024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,COXI,BUSTEDS-MH,21,510,5.000000e-01,0.0125,False,24295.058286,0.701211,1.306716,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,4.371442
5,ENCenv,BUSTED,23,500,5.000000e-01,0.0125,False,13745.467295,1.203131,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ENCenv,BUSTED-MH,23,500,5.000000e-01,0.0125,False,13749.208238,1.278712,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.086759,0.000000,0.000000
7,ENCenv,BUSTEDS,23,500,5.000000e-01,0.0125,False,13699.103102,1.254676,0.893186,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,ENCenv,BUSTEDS-MH,23,500,5.000000e-01,0.0125,False,13705.123029,1.282742,0.891511,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012449,0.000000,0.000000
9,HIV_RT,BUSTED,476,335,0.000000e+00,0.0125,True,53749.794833,1.350796,NaN,...,35|36|39|48|49|64|65|69|75|83|98|100|103|104|1...,16.0,35|48|64|75|104|122|151|162|163|174|181|188|20...,84.0,6|11|20|32|35|36|39|40|41|43|48|49|60|64|65|69...,214.0,1|3|4|5|7|9|10|12|13|14|15|16|17|19|21|22|25|2...,NaN,NaN,NaN
10,HIV_RT,BUSTED-MH,476,335,0.000000e+00,0.0125,True,53757.215664,1.404597,NaN,...,35|36|39|48|49|60|64|65|69|75|83|98|100|103|10...,12.0,48|64|75|122|151|162|174|181|188|215|228|245,85.0,6|11|20|32|35|36|39|40|41|43|48|49|60|64|65|69...,199.0,1|3|4|5|7|9|10|12|13|14|15|16|19|21|25|26|27|2...,0.000000,0.000000,0.000000


In [86]:
df.columns

Index(['Gene', 'Method', 'Sequences', 'Codons', 'LRT p-value',
       'Bonferroni p-value', 'Bonferroni significant', 'cAIC', 'CV(omega)',
       'CV(alpha)', 'w1', 'p1', 'w2', 'p2', 'w3', 'p3', 'SRV1', 'SRV_p1',
       'SRV2', 'SRV_p2', 'SRV3', 'SRV_p3', 'NUM_ER_SITES_CONSTRAINED',
       'ER_SITES_CONSTRAINED', 'NUM_ER_SITES_OPTIMIZED_NULL',
       'ER_SITES_OPTIMIZED_NULL', 'NUM_ER_SITES_CONSTRAINED_loose',
       'ER_SITES_CONSTRAINED_loose', 'NUM_ER_SITES_OPTIMIZED_NULL_loose',
       'ER_SITES_OPTIMIZED_NULL_loose', 'DH_Rate', 'TH_Rate', 'TH_Rate_SI'],
      dtype='object')

## Process

In [87]:
#for index, row in tqdm(df.iterrows()):
#    print(index)


# Set of genes

gene_set = set(df["Gene"])

models = ["BUSTED", "BUSTEDS", "BUSTED-MH", "BUSTEDS-MH"]

counts_dict = {}

for model in models:
    counts_dict[model] = {"count": 0, 
                          "countStrongPreference": 0, # delta cAIC of more than 5.
                          "genes":[],
                         "genesStrongPreference":[],
                         "median": 0}

dfh_holder = []

for gene in gene_set:
    dfh = df[df["Gene"] == gene] # df holder for each gene
    #dfh["ΔcAIC"] = ""
    cAIC_values = []
    
    for method in models:
        cAIC_val = float(dfh[dfh["Method"] == method]["cAIC"])
        cAIC_values.append(cAIC_val)
    #end for
    
    best_model_cAIC = min(cAIC_values)
    
    # Which method does this correspond to?
    best_model = ""
    
    strong_preference = False
    
    for method in models:
        cAIC_val = float(dfh[dfh["Method"] == method]["cAIC"])
        if cAIC_val == best_model_cAIC:
            best_model = method
        else:
            delta_cAIC = cAIC_val - best_model_cAIC
            if delta_cAIC > 5:
                strong_preference = True
            #end if
        #end if
    #end for
    
    print("# Gene:", gene, "Best model cAIC:", best_model_cAIC, best_model)
    
    # counts
    counts_dict[best_model]["count"] += 1
    counts_dict[best_model]["genes"].append(gene)
    
    if strong_preference == True:
        counts_dict[best_model]["countStrongPreference"] += 1
        counts_dict[best_model]["genesStrongPreference"].append(gene)
    
    #delta cAIC
    #for method in models:
    #    delta_cAIC = 0
    #    cAIC_val = float(dfh[dfh["Method"] == method]["cAIC"])
    #    delta_cAIC = cAIC_val - best_model_cAIC
    #    print("Method:", method, "Delta cAIC:", delta_cAIC)
    #    dfh[dfh["Method"] == method]["ΔcAIC"] = delta_cAIC
    
    for index, row in dfh.iterrows():
        dfh_gene = row["Gene"]
        #dfh_gene = row["Gene"]
        cAIC_val = row["cAIC"]
        delta_cAIC = cAIC_val - best_model_cAIC
        dfh.at[index, "ΔcAIC"] = delta_cAIC

    # Strong model preference
    
    
        
    dfh_holder.append(dfh)
#end for
    
    

# Gene: lysozyme Best model cAIC: 2145.579372119591 BUSTEDS
# Gene: yokoyama.rh1.cds.mod.1-990 Best model cAIC: 25895.72154682217 BUSTEDS-MH
# Gene: HepatitisD Best model cAIC: 10423.69306414134 BUSTEDS-MH
# Gene: InfluenzaA Best model cAIC: 23230.19555299281 BUSTEDS-MH
# Gene: COXI Best model cAIC: 24288.10121234685 BUSTEDS
# Gene: camelid Best model cAIC: 33668.70141504951 BUSTEDS-MH
# Gene: flavNS5 Best model cAIC: 18488.57869611563 BUSTEDS-MH
# Gene: lysin Best model cAIC: 8761.171229912901 BUSTEDS
# Gene: bglobin Best model cAIC: 7384.827434932465 BUSTEDS-MH
# Gene: HIV_RT Best model cAIC: 52026.76925371627 BUSTEDS
# Gene: VOI_SPIKE Best model cAIC: 17807.6495224851 BUSTEDS
# Gene: ENCenv Best model cAIC: 13699.10310242903 BUSTEDS
# Gene: HIVvif Best model cAIC: 6913.086831262467 BUSTEDS-MH
# Gene: adh Best model cAIC: 9354.009509023232 BUSTEDS


c:\python39\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
c:\python39\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [88]:
result = pd.concat(dfh_holder)
result

,Gene,Method,Sequences,Codons,LRT p-value,Bonferroni p-value,Bonferroni significant,cAIC,CV(omega),CV(alpha),...,NUM_ER_SITES_OPTIMIZED_NULL,ER_SITES_OPTIMIZED_NULL,NUM_ER_SITES_CONSTRAINED_loose,ER_SITES_CONSTRAINED_loose,NUM_ER_SITES_OPTIMIZED_NULL_loose,ER_SITES_OPTIMIZED_NULL_loose,DH_Rate,TH_Rate,TH_Rate_SI,ΔcAIC
49,lysozyme,BUSTED,19,130,3.769792e-01,0.0125,False,2165.851278,1.132226,NaN,...,0.0,NaN,34.0,2|14|15|17|21|23|29|37|41|47|50|62|67|69|75|78...,92.0,1|3|4|5|6|7|8|9|10|11|12|13|15|16|18|19|20|22|...,NaN,NaN,NaN,20.271906
50,lysozyme,BUSTED-MH,19,130,3.666706e-01,0.0125,False,2172.051674,1.414214,NaN,...,0.0,NaN,34.0,2|14|15|17|21|23|29|37|41|47|50|62|67|69|75|78...,84.0,1|4|5|7|8|9|10|11|12|13|15|16|18|19|20|22|24|2...,0.000000,0.000000,0.000000,26.472302
51,lysozyme,BUSTEDS,19,130,5.000000e-01,0.0125,False,2145.579372,0.368839,0.964383,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
52,lysozyme,BUSTEDS-MH,19,130,5.000000e-01,0.0125,False,2151.860776,1.284331,0.964502,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,6.281404
53,yokoyama.rh1.cds.mod.1-990,BUSTED,38,330,9.992007e-16,0.0125,True,26416.479399,1.401356,NaN,...,13.0,14|19|46|50|96|165|195|210|213|217|236|260|277,111.0,7|8|11|13|14|16|17|19|26|32|33|35|36|37|38|39|...,194.0,1|2|4|8|9|14|19|20|22|23|25|26|27|29|30|31|33|...,NaN,NaN,NaN,520.757852
54,yokoyama.rh1.cds.mod.1-990,BUSTED-MH,38,330,2.097705e-05,0.0125,True,26382.429880,1.395528,NaN,...,1.0,14,107.0,7|8|11|13|14|16|17|19|26|32|33|35|36|37|38|39|...,179.0,1|2|4|8|10|14|19|20|22|23|25|26|27|30|31|33|34...,0.270032,0.000000,12.207943,486.708333
55,yokoyama.rh1.cds.mod.1-990,BUSTEDS,38,330,1.146392e-06,0.0125,True,25924.019274,1.371073,1.023342,...,7.0,14|19|93|96|236|271|277,109.0,7|8|11|13|14|16|17|19|26|32|33|35|36|37|39|42|...,169.0,1|2|5|8|9|10|12|13|14|15|19|20|21|22|26|27|29|...,NaN,NaN,NaN,28.297728
56,yokoyama.rh1.cds.mod.1-990,BUSTEDS-MH,38,330,9.641831e-02,0.0125,False,25895.721547,1.360060,1.036956,...,0.0,NaN,109.0,7|8|11|13|14|16|17|19|26|32|33|35|36|37|39|42|...,154.0,2|8|10|13|14|15|18|19|20|21|22|26|27|29|30|31|...,0.237679,0.282681,11.410864,0.000000
17,HepatitisD,BUSTED,33,196,0.000000e+00,0.0125,True,10697.519123,1.338044,NaN,...,16.0,6|9|13|23|24|28|31|35|90|117|122|140|145|150|1...,71.0,3|4|5|6|7|8|9|11|13|16|17|20|23|24|26|28|30|31...,118.0,1|2|4|6|8|9|13|17|18|23|24|25|28|30|31|35|36|3...,NaN,NaN,NaN,273.826059
18,HepatitisD,BUSTED-MH,33,196,1.849457e-03,0.0125,True,10677.994529,1.354563,NaN,...,0.0,NaN,76.0,3|4|5|6|7|8|9|11|13|16|17|20|23|24|26|27|28|30...,105.0,1|2|3|4|5|6|7|9|13|14|18|23|24|25|28|31|33|35|...,0.252173,0.000000,0.000000,254.301465


In [91]:
delta_cAIC = result[result["ΔcAIC"] > 0]
BUSTEDS_preferred = delta_cAIC[delta_cAIC["Method"] == "BUSTEDS"]
BUSTEDSMH_preferred = delta_cAIC[delta_cAIC["Method"] == "BUSTEDS-MH"]

print("# Number of genes preferring BUSTEDS (by any amount, cAIC):", BUSTEDS_preferred.shape[0])
print("# Number of genes preferring BUSTEDS-MH (by any amount, cAIC):", BUSTEDSMH_preferred.shape[0])

y = BUSTEDSMH_preferred["ΔcAIC"].median()
print("# Median ΔcAIC genes preferring BUSTEDS-MH:", round(y, 1))

Q3 = np.quantile(BUSTEDSMH_preferred["ΔcAIC"], 0.75)
Q1 = np.quantile(BUSTEDSMH_preferred["ΔcAIC"], 0.25)
IQR = Q3 - Q1
print("# IQR (ΔcAIC):", round(IQR, 1))

z = BUSTEDS_preferred["ΔcAIC"].median()
print("# Median ΔcAIC genes preferring BUSTEDS:", round(z, 1))

Q3 = np.quantile(BUSTEDS_preferred["ΔcAIC"], 0.75)
Q1 = np.quantile(BUSTEDS_preferred["ΔcAIC"], 0.25)
IQR = Q3 - Q1
print("# IQR (ΔcAIC):", round(IQR, 1))

# ER Analysis
a = BUSTEDSMH_preferred["NUM_ER_SITES_OPTIMIZED_NULL"].median()
a_1 = BUSTEDSMH_preferred["NUM_ER_SITES_OPTIMIZED_NULL"].min()
a_2 = BUSTEDSMH_preferred["NUM_ER_SITES_OPTIMIZED_NULL"].max()
b = BUSTEDS_preferred["NUM_ER_SITES_OPTIMIZED_NULL"].median()
b_1 = BUSTEDS_preferred["NUM_ER_SITES_OPTIMIZED_NULL"].min()
b_2 = BUSTEDS_preferred["NUM_ER_SITES_OPTIMIZED_NULL"].max()

print("# Median ER Sites genes preferring BUSTEDS-MH:", round(a, 1))
print("# Minimum ER Sites genes preferring BUSTEDS-MH:", round(a_1, 1))
print("# Minimum ER Sites genes preferring BUSTEDS-MH:", round(a_2, 1))

Q3 = np.quantile(BUSTEDSMH_preferred["NUM_ER_SITES_OPTIMIZED_NULL"], 0.75)
Q1 = np.quantile(BUSTEDSMH_preferred["NUM_ER_SITES_OPTIMIZED_NULL"], 0.25)
IQR = Q3 - Q1
print("# IQR (ΔcAIC):", round(IQR, 1))

print("# Median ER Sites genes preferring BUSTEDS:", round(b, 1))
print("# Minimum ER Sites genes preferring BUSTEDS-MH:", round(b_1, 1))
print("# Minimum ER Sites genes preferring BUSTEDS-MH:", round(b_2, 1))

Q3 = np.quantile(BUSTEDS_preferred["NUM_ER_SITES_OPTIMIZED_NULL"], 0.75)
Q1 = np.quantile(BUSTEDS_preferred["NUM_ER_SITES_OPTIMIZED_NULL"], 0.25)
IQR = Q3 - Q1
print("# IQR (ΔcAIC):", round(IQR, 1))


print()

a = BUSTEDSMH_preferred["NUM_ER_SITES_OPTIMIZED_NULL_loose"].median()
a_1 = BUSTEDSMH_preferred["NUM_ER_SITES_OPTIMIZED_NULL_loose"].min()
a_2 = BUSTEDSMH_preferred["NUM_ER_SITES_OPTIMIZED_NULL_loose"].max()
b = BUSTEDS_preferred["NUM_ER_SITES_OPTIMIZED_NULL_loose"].median()
b_1 = BUSTEDS_preferred["NUM_ER_SITES_OPTIMIZED_NULL_loose"].min()
b_2 = BUSTEDS_preferred["NUM_ER_SITES_OPTIMIZED_NULL_loose"].max()

print("# Median ER Sites genes preferring BUSTEDS-MH _loose:", round(a, 1))
print("# Minimum ER Sites genes preferring BUSTEDS-MH _loose:", round(a_1, 1))
print("# Maximum ER Sites genes preferring BUSTEDS-MH _loose:", round(a_2, 1))

Q3 = np.quantile(BUSTEDSMH_preferred["NUM_ER_SITES_OPTIMIZED_NULL_loose"], 0.75)
Q1 = np.quantile(BUSTEDSMH_preferred["NUM_ER_SITES_OPTIMIZED_NULL_loose"], 0.25)
IQR = Q3 - Q1
print("# IQR (ΔcAIC) _loose:", round(IQR, 1))

print("# Median ER Sites genes preferring BUSTEDS _loose:", round(b, 1))
print("# Minimum ER Sites genes preferring BUSTEDS _loose:", round(b_1, 1))
print("# Maximum ER Sites genes preferring BUSTEDS _loose:", round(b_2, 1))

Q3 = np.quantile(BUSTEDS_preferred["NUM_ER_SITES_OPTIMIZED_NULL_loose"], 0.75)
Q1 = np.quantile(BUSTEDS_preferred["NUM_ER_SITES_OPTIMIZED_NULL_loose"], 0.25)
IQR = Q3 - Q1
print("# IQR (ΔcAIC)_loose:", round(IQR, 1))

# Number of genes preferring BUSTEDS (by any amount, cAIC): 7
# Number of genes preferring BUSTEDS-MH (by any amount, cAIC): 7
# Median ΔcAIC genes preferring BUSTEDS-MH: 6.3
# IQR (ΔcAIC): 3.1
# Median ΔcAIC genes preferring BUSTEDS: 25.4
# IQR (ΔcAIC): 22.7
# Median ER Sites genes preferring BUSTEDS-MH: 1.5
# Minimum ER Sites genes preferring BUSTEDS-MH: 0.0
# Minimum ER Sites genes preferring BUSTEDS-MH: 3.0
# IQR (ΔcAIC): nan
# Median ER Sites genes preferring BUSTEDS: 5.0
# Minimum ER Sites genes preferring BUSTEDS-MH: 0.0
# Minimum ER Sites genes preferring BUSTEDS-MH: 28.0
# IQR (ΔcAIC): 7.0

# Median ER Sites genes preferring BUSTEDS-MH _loose: 142.5
# Minimum ER Sites genes preferring BUSTEDS-MH _loose: 62.0
# Maximum ER Sites genes preferring BUSTEDS-MH _loose: 579.0
# IQR (ΔcAIC) _loose: nan
# Median ER Sites genes preferring BUSTEDS _loose: 127.0
# Minimum ER Sites genes preferring BUSTEDS _loose: 57.0
# Maximum ER Sites genes preferring BUSTEDS _loose: 201.0
# IQR (ΔcAIC)_

## ER Analysis

In [67]:
# NUM_ER_SITES_OPTIMIZED_NULL



In [50]:
result.columns

Index(['Gene', 'Method', 'Sequences', 'Codons', 'LRT p-value',
       'Bonferroni p-value', 'Bonferroni significant', 'cAIC', 'CV(omega)',
       'CV(alpha)', 'w1', 'p1', 'w2', 'p2', 'w3', 'p3', 'SRV1', 'SRV_p1',
       'SRV2', 'SRV_p2', 'SRV3', 'SRV_p3', 'NUM_ER_SITES_CONSTRAINED',
       'ER_SITES_CONSTRAINED', 'NUM_ER_SITES_OPTIMIZED_NULL',
       'ER_SITES_OPTIMIZED_NULL', 'DH_Rate', 'TH_Rate', 'TH_Rate_SI', 'ΔcAIC'],
      dtype='object')

In [55]:
counts_dict

{'BUSTED': {'count': 0,
  'countStrongPreference': 0,
  'genes': [],
  'genesStrongPreference': []},
 'BUSTEDS': {'count': 7,
  'countStrongPreference': 7,
  'genes': ['lysozyme',
   'COXI',
   'lysin',
   'HIV_RT',
   'VOI_SPIKE',
   'ENCenv',
   'adh'],
  'genesStrongPreference': ['lysozyme',
   'COXI',
   'lysin',
   'HIV_RT',
   'VOI_SPIKE',
   'ENCenv',
   'adh']},
 'BUSTED-MH': {'count': 0,
  'countStrongPreference': 0,
  'genes': [],
  'genesStrongPreference': []},
 'BUSTEDS-MH': {'count': 7,
  'countStrongPreference': 7,
  'genes': ['yokoyama.rh1.cds.mod.1-990',
   'HepatitisD',
   'InfluenzaA',
   'camelid',
   'flavNS5',
   'bglobin',
   'HIVvif'],
  'genesStrongPreference': ['yokoyama.rh1.cds.mod.1-990',
   'HepatitisD',
   'InfluenzaA',
   'camelid',
   'flavNS5',
   'bglobin',
   'HIVvif']}}

In [12]:
dfh

,Gene,Method,Sequences,Codons,LRT p-value,Bonferroni p-value,Bonferroni significant,cAIC,CV(omega),CV(alpha),...,SRV_p2,SRV3,SRV_p3,NUM_ER_SITES_CONSTRAINED,ER_SITES_CONSTRAINED,NUM_ER_SITES_OPTIMIZED_NULL,ER_SITES_OPTIMIZED_NULL,DH_Rate,TH_Rate,TH_Rate_SI
29,adh,BUSTED,23,254,0.000262,0.0125,True,9371.692413,1.085644,NaN,...,NaN,NaN,NaN,24.0,2|4|35|39|49|57|68|69|81|98|133|134|163|165|16...,2.0,163|170,NaN,NaN,NaN
30,adh,BUSTED-MH,23,254,0.021074,0.0125,False,9377.068424,1.394360,NaN,...,NaN,NaN,NaN,24.0,2|4|35|39|49|57|68|69|81|98|133|134|163|165|16...,0.0,NaN,0.087721,0.000000,0.000000
31,adh,BUSTEDS,23,254,0.001816,0.0125,True,9354.009509,1.383019,0.574009,...,0.506222,2.047467,0.225235,18.0,4|6|35|39|46|49|68|69|133|134|163|165|166|170|...,1.0,170,NaN,NaN,NaN
32,adh,BUSTEDS-MH,23,254,0.019489,0.0125,False,9359.111669,1.383861,0.552316,...,0.251789,2.808413,0.074966,15.0,6|35|39|49|69|133|134|163|165|166|170|197|216|...,0.0,NaN,0.000000,0.033096,3.364916


## End of file

In [ ]:
#for index, row in tqdm(df.iterrows()):
#    print(index)


# Set of genes

gene_set = set(df["Gene"])

models = ["BUSTED", "BUSTEDS", "BUSTED-MH", "BUSTEDS-MH"]

counts_dict = {}

for model in models:
    counts_dict[model] = 0
    
#counts_dict = {"BUSTED": 0,
#              "BUSTEDS": 0,
#              "BUSTEDS-MH": 0,
#              "BUSTED-MH": 0}

dfh_holder = []

models = []

for gene in gene_set:
    dfh = df[df["Gene"] == gene] # df holder for each gene
    
    cAIC_values = []
    
    for method in models:
        cAIC_val = float(dfh[dfh["Method"] == method]["cAIC"])
        cAIC_values.append(cAIC_val)
        counts_dict[method].update({"Genes": })
    #BUSTED_cAIC     = float(dfh[dfh["Method"] == "BUSTED"]["cAIC"])    
    #BUSTEDS_cAIC    = float(dfh[dfh["Method"] == "BUSTEDS"]["cAIC"])    
    #BUSTED_MH_cAIC  = float(dfh[dfh["Method"] == "BUSTED-MH"]["cAIC"])    
    #BUSTEDS_MH_cAIC = float(dfh[dfh["Method"] == "BUSTEDS-MH"]["cAIC"])    
    
    #best_model_cAIC = min(BUSTED_cAIC, BUSTEDS_cAIC, BUSTED_MH_cAIC ,BUSTEDS_MH_cAIC)
    
    best_model_cAIC = min(cAIC_values)
    
    # Which method does this correspond to?
    best_model = ""
    
    if BUSTED_cAIC == best_model_cAIC:
        best_model = "BUSTED"
    elif BUSTEDS_cAIC == best_model_cAIC:
        best_model = "BUSTEDS"
    elif BUSTED_MH_cAIC == best_model_cAIC:
        best_model = "BUSTED-MH"
    elif BUSTEDS_MH_cAIC == best_model_cAIC:
        best_model = "BUSTEDS-MH"
    else:
        pass
    #end if
    
    print("# Gene:", gene, "Best model cAIC:", best_model_cAIC, best_model)
    
    # counts
    counts_dict[best_model]["count"] += 1
    
    #delta cAIC
    
    for cAIC_value in 
    
    
    

In [ ]:
#for index, row in tqdm(df.iterrows()):
#    print(index)


# Set of genes

gene_set = set(df["Gene"])

models = ["BUSTED", "BUSTEDS", "BUSTED-MH", "BUSTEDS-MH"]

counts_dict = {}

for model in models:
    counts_dict[model] = {"count": 0, "genes":[]}
    
#counts_dict = {"BUSTED": 0,
#              "BUSTEDS": 0,
#              "BUSTEDS-MH": 0,
#              "BUSTED-MH": 0}

dfh_holder = []

models = []

for gene in gene_set:
    dfh = df[df["Gene"] == gene] # df holder for each gene
    
    BUSTED_cAIC     = float(dfh[dfh["Method"] == "BUSTED"]["cAIC"])    
    BUSTEDS_cAIC    = float(dfh[dfh["Method"] == "BUSTEDS"]["cAIC"])    
    BUSTED_MH_cAIC  = float(dfh[dfh["Method"] == "BUSTED-MH"]["cAIC"])    
    BUSTEDS_MH_cAIC = float(dfh[dfh["Method"] == "BUSTEDS-MH"]["cAIC"])    
    
    best_model_cAIC = min(BUSTED_cAIC, BUSTEDS_cAIC, BUSTED_MH_cAIC ,BUSTEDS_MH_cAIC)
    
    # Which method does this correspond to?
    best_model = ""
    
    if BUSTED_cAIC == best_model_cAIC:
        best_model = "BUSTED"
    elif BUSTEDS_cAIC == best_model_cAIC:
        best_model = "BUSTEDS"
    elif BUSTED_MH_cAIC == best_model_cAIC:
        best_model = "BUSTED-MH"
    elif BUSTEDS_MH_cAIC == best_model_cAIC:
        best_model = "BUSTEDS-MH"
    else:
        pass
    #end if
    
    print("# Gene:", gene, "Best model cAIC:", best_model_cAIC, best_model)
    
    # counts
    counts_dict[best_model]["count"] += 1
    counts_dict[best_model]["genes"].append(gene)
    
    #delta cAIC
    
    
    
    